이번 주에는 총 3가지를 배운다.  

1. Variable
2. Funtion
3. Backpropagation

# Variable

이번 강의를 이어나갈 때, 모든 변수(데이터)를 저장하는 형태인듯

In [91]:
import numpy as np

In [92]:
def my_print(x):
    print(f"The type : {type(x)}")
    if isinstance(x,Variable):
        print(f"The data : {x.data}")
    else:
        print(f"The data : {x}")

In [93]:
class Variable(object):
    def __init__(self, x):
        self.data = x

In [94]:
x = Variable(np.array(3.0))

my_print(x)

The type : <class '__main__.Variable'>
The data : 3.0


# Function

가상화같은 느낌으로 이해하면 될 듯  
  
직접적으로 Function class를 사용하는 것이 아니라,  
모든 함수는 Function을 항상 상속하여, 구현되도록 함

In [95]:
class Function(object):
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output
    
    def forward(self, x):
        raise NotImplementedError() # raise와 assert차이 알아내기

In [96]:
#f = Function()
#output = f(x)

In [97]:
class Square(Function):
    def forward(self, x):
        return  x ** 2
    
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

In [98]:
x = Variable(np.array(10.))
f = Square()
y = f(x)

my_print(y)

The type : <class '__main__.Variable'>
The data : 100.0


In [99]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
my_print(y)

The type : <class '__main__.Variable'>
The data : 1.648721270700128


## Magic method & Decorator

call, init과 같은 유용한 magic method 및 데코레이터 조사

### Construction and Initialization
1. __new__
2. __init__
3. __del__

In [100]:
class Myclass(object):
    def __new__(cls, *args, **kwargs):
        # This method is used for ... I don't know well when we can use  and why we should use...
        # The point is this method is called before init method
        print("__new__ method called")
        inst = super().__new__(cls)
        return inst
    
    def __init__(self, name):
        print("__init__ method called")
        self.name = name

    def __del__(self):
        # This method is called, when you use "del ~~"
        print("__del__ method called")

In [101]:
myclass = Myclass("EunNam")

print(f"name: {myclass.name}")
del myclass

try:
    print(myclass.name)
except Exception as e:
    print(f"There is no myclass: {e}")

__new__ method called
__init__ method called
name: EunNam
__del__ method called
There is no myclass: name 'myclass' is not defined


### Representing

1. __repr__
2. __str__
3. __hash__

In [109]:
class Myclass(object):    
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        # It provides informations for re-generating class instance
        return f"Myclass('{self.name}')"

    def __str__(self):
        # print(instance)
        return f"Myclass named {self.name}"
    
    def __hash__(self):
        """
        The hash function is responsible for converting the input into a shorter, constant-length output. 
        At this time, it is basic to design the output to be different when the input is different. 
        It is mainly used to check the identity of two objects.
        """
        return hash((self.name))
    
    def __eq__(self, other):
        # obj1 == obj2 behavior
        return self.__hash__() == other.__hash__()

In [110]:
myclass1 = Myclass("EunNam")
myclass2 = Myclass("Cho")

print(myclass1)

if myclass1 == myclass2:
    print("Same")
else:
    print("differnt")

Myclass named EunNam
differnt


In [111]:
myclass1

Myclass('EunNam')

### Accessing Attribute

I think these methods is not used well.  
However, when we must keep class security, Must use these.  
It seems relevant to *__"Encapsulation"__* 
  
1. __getattr__
2. __getattribution__
3. __setattr__
4. __delattr__   

========================

1. __setitem__
2. __getitem__
3. __delitem__
4. __contains__

In [112]:
class Myclass(object):    
    def __init__(self, name):
        self.name = name

    def __getattribution__(self, name):
        print(f"Rquested attribute '{name}' does not exists.")

    def __setattribution__(self, name):
        print(f"Setting attribute '{name}' to '{name}'")
        self.__dict__[name] = name

# Backpropagation
  
역전파는 다음 시간부터.  
  
엄밀히 말하면, 역전파의 장점을 부각시키기 위해 미분을 간단히 구현  
컴퓨터로 lim 보낼 수 없으니, **_수치미분_** 으로 미분
  
수치 미분은 lim 대신, **_아주 작은 값_** 을 대입하는 것  

In [106]:
def numerical_diff(f, x, eps=1e-4):
    x1 = Variable(x.data - eps)
    x2 = Variable(x.data + eps)

    y1 = f(x1)
    y2 = f(x2)

    return (y2.data - y1.data) / (2 * eps)

In [107]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)

my_print(dy)

The type : <class 'numpy.float64'>
The data : 4.000000000004


In [108]:
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f,x)

my_print(dy)

The type : <class 'numpy.float64'>
The data : 3.2974426293330694
